# 常用基本计算

在日常科研学习工作中有一些计算，比如统计计算，是很常遇到的，这里日常积累做些记录以备后用。

## NSE

NSE 全名 Nash–Sutcliffe model efficiency coefficient，用来评价水文模型的预测能力，原文可以参考：[River flow forecasting through conceptual models part I — A discussion of principles](https://www.sciencedirect.com/science/article/pii/0022169470902556)。这里根据维基百科[Nash–Sutcliffe model efficiency coefficient](https://en.wikipedia.org/wiki/Nash%E2%80%93Sutcliffe_model_efficiency_coefficient)做一些记录。

先上公式：
$$NSE=1-\frac{\sum _{t=1}^{T} (Q_m^t - Q_m^t)^2}{\sum _{t=1}^T (Q_o^t -\overline{Q_o})^2}$$
$Q_o$是观测径流平均值，$Q_m$是模拟径流。$Q_o^t$是t时段径流。

NSE值范围$-\infty$至1，NSE=1 表示模拟值和观测值非常匹配。NSE=0则表明模型预测和观测均值一致。而NSE为负数则说明观测均值都比模拟值效果好，即模拟与观测的residual variance 比 径流本身的方差还大。总之，越接近1，说明预测越好。0.5<NSE<0.65可以算是一个足够好的预测NSE的范围。

当模拟是回归时，NSE和$R^2$等价。

NSE对极值比较敏感，因此离群值影响较大。

纳什效率可以用来定量描述模型输出的准确性，而不仅仅是流量。只要有观测数据，并与模型结果进行比较，该指标可用于描述其他模型的预测精度。

下面给出二维的NSE计算方式。每行表示不同的时间，每列表示不同的站点。

In [3]:
import numpy as np
sim = np.random.rand(6,4)
sim

array([[0.72209182, 0.21352318, 0.62410322, 0.389707  ],
       [0.89507492, 0.75888337, 0.34669306, 0.40842543],
       [0.14654104, 0.46812356, 0.13435416, 0.76889576],
       [0.31079085, 0.84641048, 0.76140257, 0.94652568],
       [0.09716227, 0.27537252, 0.67535365, 0.50391768],
       [0.87018306, 0.82371468, 0.15580497, 0.22831284]])

In [4]:
obs = np.random.rand(6,4)
obs

array([[0.55331267, 0.48772107, 0.26298621, 0.86993086],
       [0.83876898, 0.31892878, 0.07505383, 0.47556347],
       [0.91504126, 0.53631456, 0.79054552, 0.52907441],
       [0.00748106, 0.1855167 , 0.32326133, 0.69029544],
       [0.50406595, 0.48948671, 0.73682538, 0.59250742],
       [0.16226021, 0.92901116, 0.45750459, 0.05923551]])

先求每个站点的观测均值，注意尽量不要用循环，for会使运算变慢。

In [6]:
obs_mean=np.mean(obs,axis=0)
obs_mean

array([0.49682169, 0.49116316, 0.44102948, 0.53610118])

然后扩展为同样shape的数组，以做减法运算（也可以广播）

In [7]:
obs_means=np.tile(obs_mean, (obs.shape[0],1))
obs_means

array([[0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118]])

接着做减法并求平方，然后每列各自求和：

In [16]:
denominator=np.sum((obs-obs_means)**2, axis=0)
denominator

array([0.64646491, 0.31684862, 0.38943503, 0.36951484])

分子部分先计算obs和sim的差值，然后平方，并每列各自求和：

In [17]:
numerator = np.sum((obs-sim)**2, axis=0)
numerator

array([1.38097153, 0.76710736, 0.92154964, 0.39472597])

然后是element-wise相除：

In [18]:
quotient = numerator/denominator
quotient

array([2.13618948, 2.42105318, 2.36637586, 1.06822765])

构造一个1向量，然后相减即可

In [21]:
my_ones=np.ones(obs.shape[1])
NSE=my_ones-quotient
NSE

array([-1.13618948, -1.42105318, -1.36637586, -0.06822765])

以上就是NSE的计算方法。